<a href="https://colab.research.google.com/github/Akash743/Mathematical-Optimization/blob/main/Problem_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install pyomo
!apt-get install -y -qq glpk-utils  #glpk is the solver
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.6 MB 5.2 MB/s 
     |████████████████████████████████| 49 kB 4.7 MB/s 
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 155680 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.1.2-2_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.1.2-2_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.1.2-2) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_4.65-1_amd64.deb ...
Unpacking libglpk40:amd64 (4.65-1) ...
Selecting previously unselected package glpk-utils.
Pr

In [7]:
model = pyo.ConcreteModel()

# Sets
model.i = pyo.RangeSet(1,3) #for no. of plants
model.j = pyo.RangeSet(1,4) #for no. of cities
# range can be useful when no. of vars is high and we don't want to name them

"""
Above is equivalent of this.
model.i = pyo.Set('Plant1','Plant2','Plannt3')
model.j = pyo.Set('City1','City2','City3','City4')
"""

# Parameters
  #Supply side(S)
model.S = pyo.Param(model.i,initialize = {1:35,2:50,3:40})
S = model.S
  #Demand side(D)
model.D = pyo.Param(model.j,initialize = {1:45,2:20,3:30,4:30})
D = model.D
  # Price
model.P = pyo.Param(model.i, model.j, initialize = {(1,1):8,(1,2):6,(1,3):10,(1,4):9, #Plant 1 ---> City 1, 2, 3, 4
                                                    (2,1):9,(2,2):12,(2,3):13,(2,4):7, #Plant 2 ---> City 1, 2, 3, 4
                                                    (3,1):14,(3,2):9,(3,3):16,(3,4):5}) #Plant 3 ---> City 1, 2, 3, 4
                                                    #since price requires both
                                                    #In other notation, could have written
                                                    #(1,1):8 ==> ("Plant1","City1"):8 and so on...
P = model.P

# Variables
model.x = pyo.Var(model.i,model.j,within = pyo.NonNegativeReals)
x = model.x

# Objective Function
def Objective_rule(model):
  return sum(sum(P[i,j]*x[i,j] for i in model.i) for j in model.j)

model.Objf = pyo.Objective(rule = Objective_rule, sense = pyo.minimize)

# Constraints 

  #Supply
def Constraint1(model,i):
  return sum(x[1,j] for j in model.j) <= S[1]
model.Const1 = pyo.Constraint(model.j, rule = Constraint1)  

def Constraint2(model,i):
  return sum(x[2,j] for j in model.j) <= S[2]
model.Const2 = pyo.Constraint(model.j, rule = Constraint2)  

def Constraint3(model,i):
  return sum(x[3,j] for j in model.j) <= S[3]
model.Const3 = pyo.Constraint(model.j, rule = Constraint3)  

  #Demand
def Constraint4(model,i):
  return sum(x[i,1] for i in model.i) >= D[1]
model.Const4 = pyo.Constraint(model.i, rule = Constraint4)  

def Constraint5(model,i):
  return sum(x[i,2] for i in model.i) >= D[2]
model.Const5 = pyo.Constraint(model.i, rule = Constraint5)  

def Constraint6(model,i):
  return sum(x[i,3] for i in model.i) >= D[3]
model.Const6 = pyo.Constraint(model.i, rule = Constraint6)  

def Constraint7(model,i):
  return sum(x[i,4] for i in model.i) >= D[4]
model.Const7 = pyo.Constraint(model.i, rule = Constraint7)  


In [9]:
Solver = SolverFactory('glpk')
results = Solver.solve(model)

print(results)
print('objective Function =', model.Objf())

for i in model.i:
  for j in model.j:
    print('Electricity sent from Plant',i,'to City',j,'=',x[i,j]())


Problem: 
- Name: unknown
  Lower bound: 1020.0
  Upper bound: 1020.0
  Number of objectives: 1
  Number of constraints: 25
  Number of variables: 13
  Number of nonzeros: 85
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.012346982955932617
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

objective Function = 1020.0
Electricity sent from Plant 1 to City 1 = 0.0
Electricity sent from Plant 1 to City 2 = 10.0
Electricity sent from Plant 1 to City 3 = 25.0
Electricity sent from Plant 1 to City 4 = 0.0
Electricity sent from Plant 2 to City 1 = 45.0
Electricity sent from Plant 2 to City 2 = 0.0
Electricity sent from Plant 2 to City 3 = 5.0
Electricity sent from Plant 2 to City 4 = 0.0
Electricity sent from Plant 3 to City 1 = 0.0
Electricity sent from Plant 3 to City 2 = 10.0
Electricity sent from Plant 